In [21]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [22]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-08 22:59:49--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.01MB/s    in 0.2s    

2022-03-08 22:59:50 (5.01 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [23]:
# Start Spark session
from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName("Hashing").getOrCreate()
spark = SparkSession.builder.appName("amazon").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [24]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_DVD_v1_00.tsv.gz"), sep='\t', header=True, inferSchema=True)

#Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...| 2015-08-31|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed|       Video DVD|          5|    

In [25]:
df.count()

5069140

In [26]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType,DateType
from pyspark.sql import Row

In [27]:
review_id_table=df.select(['review_id','customer_id','product_id','product_parent', 'review_date'])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R33UPQQUZQEM8|   27288431|B005T4ND06|     400024643| 2015-08-31|
|R3IKTNQQPD9662|   13722556|B004EPZ070|     685335564| 2015-08-31|
|R3U27V5QMCP27T|   20381037|B005S9EKCW|     922008804| 2015-08-31|
|R2TOH2QKNK4IOC|   24852644|B00FC1ZCB4|     326560548| 2015-08-31|
|R2XQG5NJ59UFMY|   15556113|B002ZG98Z0|     637495038| 2015-08-31|
|R1N1KHBRR4ZTX3|    6132474|B00X8RONBO|     896602391| 2015-08-31|
|R3OM9S0TCBP38K|   48049524|B000CEXFZG|     115883890| 2015-08-31|
|R1W4S949ZRCTBW|    3282516|B00ID8H8EW|     977932459| 2015-08-31|
|R18JL1NNQAZFV2|   51771179|B000TGJ8IU|     840084782| 2015-08-31|
|R1LP6PR06OPYUX|   31816501|B00DPMPTDS|     262144920| 2015-08-31|
| RZKBT035JA0UQ|   16164990|B00X797LUS|     883589001| 2015-08-31|
|R253N5W74SM7N3|   33386989|B00C6MXB42|     734735137| 2015-08

In [28]:
review_id_table

DataFrame[review_id: string, customer_id: int, product_id: string, product_parent: int, review_date: string]

In [29]:
review_id_table=review_id_table.withColumn('product_parent',review_id_table['product_parent'].cast(IntegerType())).withColumn('review_date',review_id_table['review_date']).withColumn('customer_id',review_id_table['customer_id'].cast(IntegerType()))
review_id_table=review_id_table.na.drop(how="any")

review_id_table=review_id_table.withColumn('review_date',review_id_table['review_date'].cast(DateType()))
review_id_table=review_id_table.dropna(how='any')
review_id_table.where(review_id_table.customer_id.isNotNull())

review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [30]:
products=df.select(['product_id','product_title'])
product=products.groupBy('product_id').count()
products=products.select(product['product_id'],'product_title')
products=products.dropDuplicates()
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001DE29SS|Chaplin (15th Ann...|
|B00ZJ2GL6G|Jurassic World 3D...|
|B00T5DM876|Pretty Little Lia...|
|B004VW4V76|         Lebanon, PA|
|B009D4SFEC|Murdoch Mysteries...|
|B000GG4Y5K|Numb3rs - The Com...|
|B00383XZP8|TCM Spotlight: Ch...|
|B00I9MS86O|Breaking Bad: The...|
|B00005K3O4|Killer Klowns Fro...|
|B00BD6KETC|     Psych: Season 7|
|B00WHZZBTQ|Heaven Adores You...|
|B000092Q5C|Remo Williams - T...|
|B004AXPWH4|10 MINUTESOLUTION...|
|B00FEP9PQG|Boardwalk Empire:...|
|B0002F6BJW|Fractured Flicker...|
|B0007LXP7W|The A-Team - Seas...|
|B005ET4NHI|  Where The Boys Are|
|B005EBOSO4|Muscle Energy Tec...|
|B003V5CTKU|Black Label Socie...|
|B000087F6L|      Fahrenheit 451|
+----------+--------------------+
only showing top 20 rows



In [31]:
customers=df.select('customer_id')

In [32]:
customers=customers.groupBy('customer_id').count()
customers=customers.withColumnRenamed('count','customer_count')
customers=customers.withColumn('customer_id',customers['customer_id'].cast(IntegerType()))
customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [33]:
vine_table=df.select(['review_id','star_rating','helpful_votes','total_votes','vine'])
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [34]:
vine_table=vine_table.withColumn('star_rating',vine_table['star_rating'].cast(IntegerType())).withColumn('helpful_votes',vine_table['helpful_votes'].cast(IntegerType())).withColumn('total_votes',vine_table['total_votes'].cast(IntegerType()))
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [35]:
mode = "append"
jdbc_url="jdbc:postgresql::5432/myPostgresDB"
config = {"user":"root", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [36]:
#review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [37]:
#products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [39]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

# New Section

# New Section